In [2]:
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
#from urllib.request import urlopen

In [3]:
word='Ῥωμαῖος'
url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
soup = bs(url.text, 'html.parser')

# finds the table
tables =soup.find_all('table')
rows =tables[0].find_all("tr")

#data is the output
data=[]

# converts the table into Array of arrays Row major, cleans any empty cells and removes latin translation
# a table contains multiple rows rows contain multiple entries
for row in rows:
    #print("ROW-----")
    for tag in row.find_all('span',{'class':'tr Latn'}):
        tag.decompose()
    
    # an array that contains the data for each row
    row_items=[]
    # for each item inside
    # row find children returns every item in a row as a na array
    
    #recursive false so you dont get repeats
    for item in row.findChildren(recursive=False):
        #print("ITEM-----")
        #print(item)
        row_items.append(item.text.replace('\n',''))
    #romve all empty cells in a row 
    while("" in row_items):
        row_items.remove("")
    data.append(row_items)
    
    
#CLEAN DATA OF PARAENTHESIS BRACKETS AND * 
for i,item in enumerate(data):
    for j,item in enumerate(data[i]):
        data[i][j] = data[i][j].replace('(','').replace(')','').replace('*','')
        

#remove notes column thats often at the bottom of the table # negative index in python start from last 
if((len(data[-1])!=len(data[-2])) or data[-1].find("note")):
    data.pop(-1)
    
    
#EVEN OUT COLUMNS
# if there is a column thats nested the split it up into new rows
if(len(data[0])!=len(data[1])):
    # if the first two rows are not equal then its likley an adjective declension chart which has wierd formatting
    #often times there are 3 columns for maxulin feminie and neuter for each colum in the first row
    #so we increase the amount of columns in the first row to match the ammount of columns in the second row
    # we do this by combining the labels
    new_row=[]
    new_row.append(data[0][0]+"/"+data[1][0])
    for i in range(1,len(data[0])):
        # if there are 3 columns for every 1 column above it then you need to itterate 3 times but theer are 9 entries in the row
        #so you have to multiply it by the ratio then round row has lenghth of 10 -1 first row is label, add 1 because 3 is not inclusive
        
        for j in range(1,round((len(data[1])-1)/(len(data[0])-1))+1):
            print(i,j)
            new_row.append(data[0][i]+"/"+data[1][j])
    #print(new_row)
    #replace first array
    data[0]= new_row
    #delete second array
    data.pop(1)
    
            
    
    
# if uneven array at the bottom seperate it usually the derrived forms adverb comparitive suplaritive 
# seperated list is likely the derrived forms of an adjective
seperated_list =[]
for i,item in enumerate(data): 
    if (i!=0):
        if(len(data[i])!=len(data[i-1])):
            seperated_list = data[i:]
            data = data[0:i]
            
            #print(data[i],"333333333333")
            break
    
#fix seperated#
if(len(seperated_list)!=0):
    if(len(seperated_list[0])==len(seperated_list[1])+1):
        seperated_list[0].pop(0)
# create index array    
index_array=[]
for i,item in enumerate(data):
    index_array.append(data[i][0])

print("--------------------")
for i in data: 
    print(i)
for i in seperated_list: 
    print(i)




1 1
1 2
1 3
2 1
2 2
2 3
3 1
3 2
3 3
--------------------
['Number/Case/Gender', 'Singular/Masculine', 'Singular/Feminine', 'Singular/Neuter', 'Dual/Masculine', 'Dual/Feminine', 'Dual/Neuter', 'Plural/Masculine', 'Plural/Feminine', 'Plural/Neuter']
['Nominative', 'Ῥωμαῖος', 'Ῥωμαίᾱ', 'Ῥωμαῖον', 'Ῥωμαίω', 'Ῥωμαίᾱ', 'Ῥωμαίω', 'Ῥωμαῖοι', 'Ῥωμαῖαι', 'Ῥωμαῖᾰ']
['Genitive', 'Ῥωμαίου', 'Ῥωμαίᾱς', 'Ῥωμαίου', 'Ῥωμαίοιν', 'Ῥωμαίαιν', 'Ῥωμαίοιν', 'Ῥωμαίων', 'Ῥωμαίων', 'Ῥωμαίων']
['Dative', 'Ῥωμαίῳ', 'Ῥωμαίᾳ', 'Ῥωμαίῳ', 'Ῥωμαίοιν', 'Ῥωμαίαιν', 'Ῥωμαίοιν', 'Ῥωμαίοις', 'Ῥωμαίαις', 'Ῥωμαίοις']
['Accusative', 'Ῥωμαῖον', 'Ῥωμαίᾱν', 'Ῥωμαῖον', 'Ῥωμαίω', 'Ῥωμαίᾱ', 'Ῥωμαίω', 'Ῥωμαίους', 'Ῥωμαίᾱς', 'Ῥωμαῖᾰ']
['Vocative', 'Ῥωμαῖε', 'Ῥωμαίᾱ', 'Ῥωμαῖον', 'Ῥωμαίω', 'Ῥωμαίᾱ', 'Ῥωμαίω', 'Ῥωμαῖοι', 'Ῥωμαῖαι', 'Ῥωμαῖᾰ']
['Adverb', 'Comparative', 'Superlative']
['Ῥωμαίως', 'Ῥωμαιότερος', 'Ῥωμαιότᾰτος']


In [4]:

df = pd.DataFrame(data[1:],columns=data[0], index= index_array[1:])
#df = pd.DataFrame(data[1:],columns=data[0])
display(df.drop(df.columns[0],axis=1))
if(len(seperated_list)!=0):
    df2=pd.DataFrame(seperated_list[1:],columns=seperated_list[0])
    display(df2)
 #display(df.iloc[[0]])
 
 



,Singular/Masculine,Singular/Feminine,Singular/Neuter,Dual/Masculine,Dual/Feminine,Dual/Neuter,Plural/Masculine,Plural/Feminine,Plural/Neuter
Nominative,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
Genitive,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων
Dative,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις
Accusative,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ
Vocative,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ


,Adverb,Comparative,Superlative
0,Ῥωμαίως,Ῥωμαιότερος,Ῥωμαιότᾰτος


In [7]:
#test2 #allot easier proably just going to use this 
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def generate_variations(word):

       
        #test words
        #'Ῥωμαῖος'
        #'μισώ'
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')

        #HEADERS ---------------------------
        #find all tables 
        tables =soup.find_all('table')
        h3 =[]
        h4 =[]
        # for each table find the preceding header and sub header
        for i,item in enumerate(tables): 
                h3.append(tables[i].find_previous('h3').text.replace('[edit]',''))
                h4.append(tables[i].find_previous('h4').text.replace('[edit]',''))
        #print(h3)
        #print(h4)
        #-------------------------
        
        #DATAFRAMES----------------------------------
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                #display(df)
                 #CREATE A FOLDER
                os.makedirs(h3[i],exist_ok=True)
                print(h3[i])
                print(h4[i])
                print('WRITE TO FILE--------------------------------')
                df=dataframes[0]
                df=df.dropna(how='all',axis=0)
                df=df.drop(list(df.filter(regex = 'Unnamed')), axis = 1)
                #df =df.loc[:,~df.columns.str.startswith('Unnamed')]
                df=df.replace({'➤':''}, regex=True)
                df = df[df.iloc[:,0].str.contains("Notes") == False]
                #df = df.style.set_properties(**{'text-align': 'middle'})
                df.to_csv(f'{h3[i]}/{word}_{h4[i]}.csv')
                display(df)
                # READ 
                print('READ FROM FILE ------------------------------------')
                display(pd.read_csv(f'{h3[i]}/{word}_{h4[i]}.csv'))
        #----------------------------------
generate_variations('Ῥωμαῖος')

        




,Number,Unnamed: 1,Singular,Singular.1,Singular.2,Unnamed: 5,Dual,Dual.1,Dual.2,Unnamed: 9,Plural,Plural.1,Plural.2,Unnamed: 13,Unnamed: 14
0,Case/Gender,NaN,Masculine,Feminine,Neuter,NaN,Masculine,Feminine,Neuter,NaN,Masculine,Feminine,Neuter,NaN,NaN
1,Nominative,NaN,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,NaN,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,NaN,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ,NaN,NaN
2,Genitive,NaN,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,NaN,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,NaN,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων,NaN,NaN
3,Dative,NaN,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,NaN,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,NaN,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις,NaN,NaN
4,Accusative,NaN,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,NaN,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,NaN,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ,NaN,NaN
5,Vocative,NaN,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,NaN,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,NaN,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ,NaN,NaN
6,Derived forms,NaN,Adverb,Adverb,Adverb,NaN,Comparative,Comparative,Comparative,NaN,Superlative,Superlative,Superlative,NaN,NaN
7,Derived forms,NaN,Ῥωμαίως,Ῥωμαίως,Ῥωμαίως,NaN,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότερος,NaN,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,NaN,NaN
8,Notes:,NaN,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...


Adjective
Inflection
WRITE TO FILE--------------------------------


,Number,Singular,Singular.1,Singular.2,Dual,Dual.1,Dual.2,Plural,Plural.1,Plural.2
0,Case/Gender,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter
1,Nominative,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
2,Genitive,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων
3,Dative,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις
4,Accusative,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ
5,Vocative,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
6,Derived forms,Adverb,Adverb,Adverb,Comparative,Comparative,Comparative,Superlative,Superlative,Superlative
7,Derived forms,Ῥωμαίως,Ῥωμαίως,Ῥωμαίως,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος


READ FROM FILE ------------------------------------


,Unnamed: 0,Number,Singular,Singular.1,Singular.2,Dual,Dual.1,Dual.2,Plural,Plural.1,Plural.2
0,0,Case/Gender,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter
1,1,Nominative,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
2,2,Genitive,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων
3,3,Dative,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις
4,4,Accusative,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ
5,5,Vocative,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
6,6,Derived forms,Adverb,Adverb,Adverb,Comparative,Comparative,Comparative,Superlative,Superlative,Superlative
7,7,Derived forms,Ῥωμαίως,Ῥωμαίως,Ῥωμαίως,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος


,Case / #,Singular,Dual,Plural,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Nominative,ὁ Ῥωμαῖος,τὼ Ῥωμαίω,οἱ Ῥωμαῖοι,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Genitive,τοῦ Ῥωμαίου,τοῖν Ῥωμαίοιν,τῶν Ῥωμαίων,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dative,τῷ Ῥωμαίῳ,τοῖν Ῥωμαίοιν,τοῖς Ῥωμαίοις,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Accusative,τὸν Ῥωμαῖον,τὼ Ῥωμαίω,τοὺς Ῥωμαίους,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vocative,Ῥωμαῖε,Ῥωμαίω,Ῥωμαῖοι,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Notes:,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...,This table gives Attic inflectional endings. F...


Noun
Inflection
WRITE TO FILE--------------------------------


,Number,Singular,Singular.1,Singular.2,Dual,Dual.1,Dual.2,Plural,Plural.1,Plural.2
0,Case/Gender,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter
1,Nominative,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
2,Genitive,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων
3,Dative,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις
4,Accusative,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ
5,Vocative,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
6,Derived forms,Adverb,Adverb,Adverb,Comparative,Comparative,Comparative,Superlative,Superlative,Superlative
7,Derived forms,Ῥωμαίως,Ῥωμαίως,Ῥωμαίως,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος


READ FROM FILE ------------------------------------


,Unnamed: 0,Number,Singular,Singular.1,Singular.2,Dual,Dual.1,Dual.2,Plural,Plural.1,Plural.2
0,0,Case/Gender,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter,Masculine,Feminine,Neuter
1,1,Nominative,Ῥωμαῖος,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
2,2,Genitive,Ῥωμαίου,Ῥωμαίᾱς,Ῥωμαίου,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίων,Ῥωμαίων,Ῥωμαίων
3,3,Dative,Ῥωμαίῳ,Ῥωμαίᾳ,Ῥωμαίῳ,Ῥωμαίοιν,Ῥωμαίαιν,Ῥωμαίοιν,Ῥωμαίοις,Ῥωμαίαις,Ῥωμαίοις
4,4,Accusative,Ῥωμαῖον,Ῥωμαίᾱν,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαίους,Ῥωμαίᾱς,Ῥωμαῖᾰ
5,5,Vocative,Ῥωμαῖε,Ῥωμαίᾱ,Ῥωμαῖον,Ῥωμαίω,Ῥωμαίᾱ,Ῥωμαίω,Ῥωμαῖοι,Ῥωμαῖαι,Ῥωμαῖᾰ
6,6,Derived forms,Adverb,Adverb,Adverb,Comparative,Comparative,Comparative,Superlative,Superlative,Superlative
7,7,Derived forms,Ῥωμαίως,Ῥωμαίως,Ῥωμαίως,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότερος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος,Ῥωμαιότᾰτος


In [6]:
# THESE ARE THE MAIN TYPES OF SNETIMENT EXPRESSIONS THAT I CAN THINK OF 
#Accusitive(plural),opinion(singular) - I hate romans -         μισώ τους Ρωμαίους
#Nomitive(plural),opinion(plural)-      romans hate us -        οι Ρωμαίοι μας μισούν
#Nomitive,adjective -                   romans are evil -       οι Ρωμαίοι είναι κακοί
#Nomitive,simile-                       romans are like sheep - οι Ρωμαίοι είναι σαν πρόβατα
#Nomitive,metaphore-                    romans are snakes -     οι Ρωμαίοι είναι φίδια
#(NEGATION)Accusitive,opinion -         I dont hate romans -    δεν μισώ τους Ρωμαίους
#(NEGATION)Nomitive,adjective           romans are not evil -   οι Ρωμαίοι δεν είναι κακοί
#complex
#i like how well mannerd the greeks are  -postive
# i dislike the fact that romans are so nice  neutral

# will short scentences be enought for longer scentences like thes maybe idk
# we could do a semi procedural approach with templates and masking 
#we only come up with a few thousand sentment scentences then 
#TWO STAGE APSECT FIRST ner identify the aspects of the scentence 
#aspect expects aspects as input it does not identify the aspects we need both
os.makedirs('datasets',exist_ok=True)

def test():
    #PRESENT TEST VERB SINGULAR + ACCUSITIVE NOUN 
    
    #verb + 
    pass

def accusitive_opinion_generator():
    #VERB + ACCUSITIVE NOUN
    pass
def nomitive_opinion_generator():
    #NOMITIVE NOUN + #VERB 
    #METAPHORES
    pass
def nomitive_simile_generator():
    pass

def nomitive_negation_generator():
    pass
def accusitive_negation_generator():
    pass